REsizing

In [2]:
from CellClicker.cell_clicker import ImageViewer
import tkinter as tk
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageViewer(root)
    root.mainloop()


In [1]:
from CellClicker.image_selector_multiphase import load_ui_from_folder

load_ui_from_folder()

Selected XML File: E:/Scott/Data/20240417\user_selections\Scott.xml
Selected XML: E:/Scott/Data/20240417\user_selections\Scott.xml
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t060.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:\Scott\Data\20240417\images\20240417_P1_t058.png
E:\Scott\Data\20240417\images\20240417_P1_t060.png
E:\Scott\Data\20240417\images\20240417_P1_t054.png
E:\Scott\Data\20240417\images\20240417_P1_t047.png
E:\Scott\Data\20240417\images\20240417_P1_t014.png
E:

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\u1604360\.conda\envs\CellToolsGPU\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "e:\Scott\GitHub\CelFDrive\CellClicker\image_selector_multiphase.py", line 60, in <lambda>
    window.protocol("WM_DELETE_WINDOW", lambda: on_window_close(window, root))
NameError: name 'on_window_close' is not defined
